In [1]:
import pandas as pd
import json
import sqlite3

In [2]:
try:
    # Reading in CSV data
    phone_df = pd.read_csv('user_behavior_dataset.csv', index_col=False)
    # Showing Input
    print("# of cols:", len(phone_df.count()))
    print("# of records:", len(phone_df.count('columns')))
    phone_df.head()
except:
    print("Something went wrong reading in the CSV data. Check your filepath.")

# of cols: 11
# of records: 700


In [3]:
try:
    # Reading in JSON Data
    with open('all_hour.geojson.json') as f:
        earthquake_json = json.load(f)
        # Showing Input
        numCols = 0
        for i in pd.json_normalize(earthquake_json['features']):
            numCols += 1
        print("# of cols:", numCols)
        print("# of records:", len(earthquake_json['features']), "\n")
        print(json.dumps(earthquake_json, indent=4))
except:
    print("Something went wrong reading in the JSON data. Check your filepath.")

# of cols: 30
# of records: 9 

{
    "type": "FeatureCollection",
    "metadata": {
        "generated": 1729468344000,
        "url": "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_hour.geojson",
        "title": "USGS All Earthquakes, Past Hour",
        "status": 200,
        "api": "1.14.1",
        "count": 9
    },
    "features": [
        {
            "type": "Feature",
            "properties": {
                "mag": 1.6,
                "place": "17 km NE of Petersville, Alaska",
                "time": 1729467261342,
                "updated": 1729467382777,
                "tz": null,
                "url": "https://earthquake.usgs.gov/earthquakes/eventpage/ak024dilwc3a",
                "detail": "https://earthquake.usgs.gov/earthquakes/feed/v1.0/detail/ak024dilwc3a.geojson",
                "felt": null,
                "cdi": null,
                "mmi": null,
                "alert": null,
                "status": "automatic",
                "tsuna

In [4]:
try:
    # Create SQLite DB
    connection = sqlite3.connect('phone_data.db')
    cursor = connection.cursor()
    cursor.execute("drop table if exists phoneData")
    cursor.execute("drop table if exists quakeData")
except:
    print("Something went wrong connecting to the SQLite database")

In [5]:
# Input first transformation output type
out_format = ""

while out_format != "SQL" and out_format != "JSON":
    out_format = input("""
Into which data structure should the CSV phone data be transformed?\n
> SQL
> JSON\n
    """).upper()


Into which data structure should the CSV phone data be transformed?

> SQL
> JSON

     sql


In [6]:
# Input second transformation output type
out_format_2 = ""

while out_format_2 != "CSV" and out_format_2 != "SQL":
    out_format_2 = input("""
Into which data structure should the JSON earthquake data be transformed?\n
> CSV
> SQL\n
    """).upper()


Into which data structure should the JSON earthquake data be transformed?

> CSV
> SQL

     sql


In [7]:
try:
    # Dropping 2 Columns
    phone_df.drop(columns=['Number of Apps Installed', 'User Behavior Class'], inplace=True)
except:
    print("Something went wrong dropping columns from the DataFrame. Make sure they haven't already been removed.") 

# Convert CSV data to JSON
if out_format == "JSON":
    try:
        phone_json = json.loads(phone_df.to_json(orient='table'))
        # Showing Output
        print("# of cols:", len(phone_json['schema']['fields']) - 1)
        print("# of records:", len(phone_json['data']), "\n")
        print(json.dumps(phone_json, indent=4))
        # Writing JSON data to file
        with open('phone.json', 'w') as f:
            json.dump(phone_json, f)
    except:
        print("Something went wrong converting the CSV to JSON or in writing the JSON file. Check your output filepath.")

# Convert CSV data to SQL table
else:
    try:
        phone_df.to_sql('phoneData', connection)
    
        # Showing Output
        numCols = 0
        for row in cursor.execute("pragma table_info(phoneData)"):
            numCols += 1
        print("# of cols:", numCols - 1)
        print("# of records:", len(cursor.execute("select * from phoneData").fetchall()), "\n")
        rows = cursor.execute("""
        select * from phoneData
        """)
        for row in rows:
            print(row)
    except:
        print("Something went wrong converting the CSV to SQL on in writing to the database. Check that the table doesn't already exist.")

# of cols: 9
# of records: 700 

(0, 1, 'Google Pixel 5', 'Android', 393, 6.4, 1872, 1122, 40, 'Male')
(1, 2, 'OnePlus 9', 'Android', 268, 4.7, 1331, 944, 47, 'Female')
(2, 3, 'Xiaomi Mi 11', 'Android', 154, 4.0, 761, 322, 42, 'Male')
(3, 4, 'Google Pixel 5', 'Android', 239, 4.8, 1676, 871, 20, 'Male')
(4, 5, 'iPhone 12', 'iOS', 187, 4.3, 1367, 988, 31, 'Female')
(5, 6, 'Google Pixel 5', 'Android', 99, 2.0, 940, 564, 31, 'Male')
(6, 7, 'Samsung Galaxy S21', 'Android', 350, 7.3, 1802, 1054, 21, 'Female')
(7, 8, 'OnePlus 9', 'Android', 543, 11.4, 2956, 1702, 31, 'Male')
(8, 9, 'Samsung Galaxy S21', 'Android', 340, 7.7, 2138, 1053, 42, 'Female')
(9, 10, 'iPhone 12', 'iOS', 424, 6.6, 1957, 1301, 42, 'Male')
(10, 11, 'Google Pixel 5', 'Android', 53, 1.4, 435, 162, 34, 'Female')
(11, 12, 'OnePlus 9', 'Android', 215, 5.5, 1690, 641, 24, 'Male')
(12, 13, 'OnePlus 9', 'Android', 462, 6.2, 2303, 1099, 57, 'Female')
(13, 14, 'Xiaomi Mi 11', 'Android', 215, 4.9, 1662, 857, 43, 'Male')
(14, 15, 'iP

In [8]:
try:
    # Convert JSON data to CSV
    earthquake_df = pd.DataFrame(pd.json_normalize(earthquake_json['features']))
except:
    print("Something went wrong converting the JSON data to CSV.")

try:
    # Dropping 5 columns
    earthquake_df.drop(columns=['properties.tz', 'properties.felt', 'properties.mmi', 'properties.cdi', 'properties.alert'], inplace=True)
except:
    print("Something went wrong dropping columns from the DataFrame. Make sure they haven't already been removed.")

if out_format_2 == "CSV":
    try:
        # Showing Output
        print("# of cols:", len(earthquake_df.count()))
        print("# of records:", len(earthquake_df.count('columns')), "\n")
        print(earthquake_df.head())
        # Writing CSV data to file
        earthquake_df.to_csv('earthquake.csv')
    except:
        print("Something went wrong writing the CSV data to file. Check your filepath.")
    
# Convert DataFrame to SQL table
else:
    try:
        earthquake_df['geometry.coordinates'] = earthquake_df['geometry.coordinates'].astype(str)
        earthquake_df.to_sql('quakeData', connection)
    
        # Showing Output
        numCols = 0
        for row in cursor.execute("pragma table_info(quakeData)"):
            numCols += 1
        print("# of cols:", numCols - 1)
        print("# of records:", len(cursor.execute("select * from quakeData").fetchall()), "\n")
        rows = cursor.execute("""
        select * from quakeData
        """)
        for row in rows:
            print(row)
    except:
        print("Something went wrong converting the DataFrame to SQL or writing to the database. Check that the table doesn't already exist.")

# of cols: 25
# of records: 9 

(0, 'Feature', 'ak024dilwc3a', 1.6, '17 km NE of Petersville, Alaska', 1729467261342, 1729467382777, 'https://earthquake.usgs.gov/earthquakes/eventpage/ak024dilwc3a', 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/detail/ak024dilwc3a.geojson', 'automatic', 0, 39, 'ak', '024dilwc3a', ',ak024dilwc3a,', ',ak,', ',origin,phase-data,', None, None, 1.19, None, 'ml', 'earthquake', 'M 1.6 - 17 km NE of Petersville, Alaska', 'Point', '[-150.5302, 62.6138, 16.9]')
(1, 'Feature', 'nc75076581', 1.02, '7 km NW of Parkfield, CA', 1729466916860, 1729467014777, 'https://earthquake.usgs.gov/earthquakes/eventpage/nc75076581', 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/detail/nc75076581.geojson', 'automatic', 0, 16, 'nc', '75076581', ',nc75076581,', ',nc,', ',nearby-cities,origin,phase-data,', 11.0, 0.01546, 0.05, 148.0, 'md', 'earthquake', 'M 1.0 - 7 km NW of Parkfield, CA', 'Point', '[-120.488670349121, 35.9396667480469, 9.86999988555908]')
(2, 'Feature', 'us